In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.chrome.options import Options 
from selenium.webdriver.common.keys import Keys
import chromedriver_binary
import gspread
import time
import cv2
import numpy as np
from PIL import Image
import requests

from oauth2client.service_account import ServiceAccountCredentials 
scope = ['https://spreadsheets.google.com/feeds','https://www.googleapis.com/auth/drive']
credentials = ServiceAccountCredentials.from_json_keyfile_name('suraping-381608-4562bacb42ee.json', scope)

def download_img(image, filename):
    r = requests.get(url=image, cookies=cookie, headers=headers, stream=True)
    if r.status_code == 200:
        with open(filename, 'wb') as f:
            f.write(r.content)

date = 1 #当日は0、前日は1・・・
date2 = "08/16"
pmc = "27126003" # 店舗ID
SPREADSHEET_KEY = '1ZfzOHjE0r1-hjUs7Klxxveag0EX1t59UqVps1cVegRA' # スプレッドシートのID
pstart = 1 # パチンコの1台目
sstart = 67 # スロットの1台目
wait = 0.01

user_id = "niwaniwa478@gmail.com"
user_pw = "niwaniwa28"

Template_p_path = "site777_2/template_p.png"
Template_s_path = "site777_2/template_s.png"
dot_point_p = 130000 / 432
dot_point_px = 60000 / 234
dot_point_s = 20000 / 432

mobile_emulation = { "deviceName": "iPhone 12 Pro" }

headers = {
    "User-Agent": "Mozilla/5.0 (iPhone; CPU iPhone OS 16_1 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) CriOS/108.0.5359.112 Mobile/15E148 Safari/604.1",
    "Referer" : "https://m.site777.jp/f/D3000.do?pmc=22021007&gc=2&dn=5019&urt=2173&mdc=120104&dsgk=0&dtdd=2"
}

options = Options()
options.add_experimental_option("mobileEmulation", mobile_emulation)
#options.add_argument('--headless')
options.binary_location = 'C:\\Program Files\\Google\\Chrome\\Application\\Chrome.exe'
chrome_service = ChromeService(executable_path='C:\\Users\\garden026\\WD\\114.0.5735.90\\chromedriver.exe')

gc = gspread.authorize(credentials)
wb = gc.open_by_key(SPREADSHEET_KEY)

worksheet = wb.worksheet("Config")
p4row = worksheet.cell(11,  23).value
p1row = worksheet.cell(11,  24).value
s20row = worksheet.cell(11,  28).value
s5row = worksheet.cell(11,  29).value
s10row = worksheet.cell(11,  30).value

browser = webdriver.Chrome(service=chrome_service,options=options)
browser.command_executor._commands["send_command"] = ("POST", '/session/$sessionId/chromium/send_command')
params = {'cmd': 'Page.setDownloadBehavior', 'params': {'behavior': 'allow', 'downloadPath': 'site777'}}
command_result = browser.execute("send_command", params)

browser.get('https://m.site777.jp/f/A0100.do')
time.sleep(1)

browser.find_element("xpath",'/html/body/section[1]/div/div/p[3]/a').click()
time.sleep(1)

browser.find_element("xpath",'/html/body/section/section[2]/div/div/p').click()
time.sleep(1)

browser.find_element("xpath",'/html/body/div[1]/div/section/div/dl/dd/form/div[1]/input').send_keys(user_id)
browser.find_element("xpath",'/html/body/div[1]/div/section/div/dl/dd/form/div[2]/input').send_keys(user_pw)
time.sleep(1)

browser.execute_script('document.fmLogin.submit()')
time.sleep(1)

browser.get('https://m.site777.jp/f/D0400.do?pmc=' + str(pmc))
time.sleep(1)

browser.find_element("xpath",'/html/body/div[2]/section/div[1]/dl/dd/form/input[1]').send_keys(pstart)
time.sleep(1)

browser.execute_script("S7Recaptcha.grv3FormSubmit(S7Recaptcha.GRV3_ACTION_HOMEPAGE,document.forms['formdaino'])")
time.sleep(1)

pmdc = browser.current_url.split("mdc=")[1]

browser.get('https://m.site777.jp/f/D0400.do?pmc=' + str(pmc))
time.sleep(1)

browser.find_element("xpath",'/html/body/div[2]/section/div[1]/dl/dd/form/input[1]').send_keys(sstart)
time.sleep(1)

browser.execute_script("S7Recaptcha.grv3FormSubmit(S7Recaptcha.GRV3_ACTION_HOMEPAGE,document.forms['formdaino'])")
time.sleep(1)

smdc = browser.current_url.split("mdc=")[1]

ck = browser.get_cookies()
k = 0
while k < 30:
    ccheck = ck[k]["name"]
    if ccheck == "PO_SP_ID":
        po_sp_id = ck[k]["value"]
        k = 30
    k = k + 1
k = 0
while k < 30:
    ccheck = ck[k]["name"]
    if ccheck == "JSESSIONID":
        jsessionid = ck[k]["value"]
        k = 30
    k = k + 1

cookie = {"PO_SP_ID" : po_sp_id,
         "JSESSIONID" : jsessionid}

browser.close()

p4table = []
p1table = []
s20table = []
s5table = []
s10table = []

# Pachinko

dn = pstart
mdc = pmdc

roop = 0
while roop < 1:
    try:
        url = "https://m.site777.jp/f/D2600S.do?pmc=" + str(pmc) + "&gc=2&dtdd=" + str(date) + "&dsgk=0&dn=" + str(dn) + "&mdc=" + str(mdc)
        response = requests.get(url=url, headers=headers, cookies=cookie)
        jsonData = response.text

        i = 1
        while i < 100:
            try:
                temp = jsonData.split("<daiNo>")[i].split("</daiNo>")[0]
                if int(dn) == int(temp):
                    m = i
                    i = 1000
                i = i + 1
            except:
                i = 1000
                continue
            else:
                pass

        cc = 0
        while cc < 30:
            try:
                dn = jsonData.split("<daiNo>")[m + cc].split("</daiNo>")[0]
                mdc = jsonData.split("<modelCode>")[m + cc].split("</modelCode>")[0]
                unit = jsonData.split("<unit>")[m + cc].split("</unit>")[0]
                bb = jsonData.split("<bonusCount>")[m + cc].split("</bonusCount>")[0]
                rb = jsonData.split("<hatariSu>")[m + cc].split("</hatariSu>")[0]

                image = "https://m.site777.jp" + jsonData.split("<bigGraphUrl>")[m + cc].split("</bigGraphUrl>")[0].replace("amp;","")

                k = 0
                while(k < 10):
                    try:
                        filename =  'site777_2/dai_'+str(dn)+'.png'
                        download_img(image, filename)

                        time.sleep(wait)

                        Target_path = filename

                        template = cv2.imread(Template_p_path)
                        template_gray = cv2.cvtColor(template, cv2.COLOR_BGR2GRAY)

                        img_rgb = cv2.imread(Target_path)
                        img_gray = cv2.cvtColor(img_rgb, cv2.COLOR_BGR2GRAY)

                        res = cv2.matchTemplate(img_gray,template_gray,cv2.TM_CCOEFF_NORMED)

                        threshold = 0.6
                        loc = np.where( res >= threshold)

                        Mergin = 5
                        index = []
                        for iii in range(len(loc[1][:])):
                            count = 0
                            for j in range(len(loc[1][:])):
                                if abs(loc[1][:][iii] - loc[1][:][j]) < Mergin and abs(loc[0][:][iii] - loc[0][:][j]) < Mergin:
                                    if count > 0:
                                        index.append(j)
                                    count = count + 1

                        loc = np.delete(loc, index, axis=1)

                        w, h = template_gray.shape[::-1]

                        cv2.imwrite("site777_2/template_result.png",img_rgb)

                        img_bin = cv2.Canny(img_gray,128,200)
                        cv2.imwrite("site777_2/bin_result.png",img_bin)

                        count = 0

                        for x in range(0,234):
                            xx = 238 - x
                            a = 9
                            b = 442
                            if xx < 53:
                                a = 248
                                b = 435
                            for y in range(a,b):
                                if y < 341 or y > 343:
                                    if all(img_bin[y, xx]==[255, 255, 255]):
                                        point = 342 - y
                                        count = count + 1
                                        break
                            if count == 1:
                                break

                        game = round(dot_point_px * ( xx - 10) , 0)
                        samai = round(dot_point_p * point ,0)-300
                        if xx==10:
                            samai = 0
                            game = 0

                        k = 10

                    except:
                        k = k + 1
                        continue

                    else:
                        pass
                    
                temp = jsonData.split("<daiNo>")[m + cc + 1].split("</daiNo>")[0]
                li = (str(date2) + "|" + str(mdc) + "|" + str(dn) + "|" + str(game) + "|" + str(samai) + "|" + str(bb) + "|" + str(rb)).split("|")
                print(li)
                if unit == "400":
                    p4table.append(li)
                if unit == "100":
                    p1table.append(li)
                cc = cc + 1

            except:
                if cc == 0:
                    li = (str(date2) + "|" + str(mdc) + "|" + str(dn) + "|" + str(game) + "|" + str(samai) + "|" + str(bb) + "|" + str(rb)).split("|")
                    print(li)
                    if unit == "400":
                        p4table.append(li)
                    if unit == "100":
                        p1table.append(li)
                    roop = 1
                dn = jsonData.split("<daiNo>")[m + cc].split("</daiNo>")[0]
                mdc = jsonData.split("<modelCode>")[m + cc].split("</modelCode>")[0]
                cc = 30
                continue
            else:
                pass
            
    except:
        continue
    else:
        pass

# Slot

dn = sstart
mdc = smdc

roop = 0
while roop < 1:
    try:
        url = "https://m.site777.jp/f/D3000S.do?pmc=" + str(pmc) + "&gc=2&dtdd=" + str(date) + "&dsgk=0&dn=" + str(dn) + "&mdc=" + str(mdc)
        response = requests.get(url=url, headers=headers, cookies=cookie)
        jsonData = response.text

        i = 1
        while i < 1000:
            try:
                temp = jsonData.split("<daiNo>")[i].split("</daiNo>")[0]
                if int(dn) == int(temp):
                    m = i
                    i = 1000
                i = i + 1
            except:
                i = 1000
                continue
            else:
                pass

        cc = 0
        while cc < 30:
            try:
                dn = jsonData.split("<daiNo>")[m + cc].split("</daiNo>")[0]
                mdc = jsonData.split("<modelCode>")[m + cc].split("</modelCode>")[0]
                unit = jsonData.split("<unit>")[m + cc].split("</unit>")[0]
                game = jsonData.split("<totalGameCount>")[m + cc].split("</totalGameCount>")[0]
                bb = jsonData.split("<bonusCount>")[m + cc].split("</bonusCount>")[0]
                rb = jsonData.split("<regularCount>")[m + cc].split("</regularCount>")[0]
                art = jsonData.split("<artCount>")[m + cc].split("</artCount>")[0]

                image = "https://m.site777.jp" + jsonData.split("<bigGraphUrl>")[m + cc].split("</bigGraphUrl>")[0].replace("amp;","")

                k = 0
                while(k < 10):
                    try:
                        filename =  'site777_2/dai_'+str(dn)+'.png'
                        download_img(image, filename)

                        time.sleep(wait)

                        Target_path = filename

                        template = cv2.imread(Template_s_path)
                        template_gray = cv2.cvtColor(template, cv2.COLOR_BGR2GRAY)

                        img_rgb = cv2.imread(Target_path)
                        img_gray = cv2.cvtColor(img_rgb, cv2.COLOR_BGR2GRAY)

                        res = cv2.matchTemplate(img_gray,template_gray,cv2.TM_CCOEFF_NORMED)

                        threshold = 0.6
                        loc = np.where( res >= threshold)

                        Mergin = 5
                        index = []
                        for iii in range(len(loc[1][:])):
                            count = 0
                            for j in range(len(loc[1][:])):
                                if abs(loc[1][:][iii] - loc[1][:][j]) < Mergin and abs(loc[0][:][iii] - loc[0][:][j]) < Mergin:
                                    if count > 0:
                                        index.append(j)
                                    count = count + 1

                        loc = np.delete(loc, index, axis=1)

                        w, h = template_gray.shape[::-1]

                        cv2.imwrite("site777_2/template_result.png",img_rgb)

                        img_bin = cv2.Canny(img_gray,128,200)
                        cv2.imwrite("site777_2/bin_result.png",img_bin)

                        count = 0

                        for x in range(0,234):
                            xx = 238 - x
                            a = 9
                            b = 442
                            if xx < 53:
                                a = 248
                                b = 435
                            for y in range(a,b):
                                if y < 332 or y > 334:
                                    if all(img_bin[y, xx]==[255, 255, 255]):
                                        point = 333 - y
                                        count = count + 1
                                        break
                            if count == 1:
                                break

                        samai = round(dot_point_s * point ,0)
                        if xx==10:
                            samai = 0

                        k = 10

                    except:
                        k = k + 1
                        continue

                    else:
                        pass

                temp = jsonData.split("<daiNo>")[m + cc + 1].split("</daiNo>")[0]
                li = (str(date2) + "|" + str(mdc) + "|" + str(dn) + "|" + str(game) + "|" + str(samai) + "|" + str(bb) + "|" + str(rb) + "|" + str(art)).split("|")
                print(li)
                if unit == "2173":
                    s20table.append(li)
                li = (str(date2) + "|" + str(mdc) + "|" + str(dn) + "|" + str(game) + "|" + str(samai) + "|" + str(bb) + "|" + str(rb) + "|" + str(art)).split("|")
                print(li)
                if unit == "561":
                    s5table.append(li)
                li = (str(date2) + "|" + str(mdc) + "|" + str(dn) + "|" + str(game) + "|" + str(samai) + "|" + str(bb) + "|" + str(rb) + "|" + str(art)).split("|")
                print(li)
                if unit == "1123":
                    s10table.append(li)
                cc = cc + 1

            except:
                if cc == 0:
                    li = (str(date2) + "|" + str(mdc) + "|" + str(dn) + "|" + str(game) + "|" + str(samai) + "|" + str(bb) + "|" + str(rb) + "|" + str(art)).split("|")
                    print(li)
                    if unit == "2173":
                        s20table.append(li)
                    li = (str(date2) + "|" + str(mdc) + "|" + str(dn) + "|" + str(game) + "|" + str(samai) + "|" + str(bb) + "|" + str(rb) + "|" + str(art)).split("|")
                    print(li)
                    if unit == "561":
                        s5table.append(li)
                    li = (str(date2) + "|" + str(mdc) + "|" + str(dn) + "|" + str(game) + "|" + str(samai) + "|" + str(bb) + "|" + str(rb) + "|" + str(art)).split("|")
                    print(li)
                    if unit == "1123":
                        s10table.append(li)
                    roop = 1
                dn = jsonData.split("<daiNo>")[m + cc].split("</daiNo>")[0]
                mdc = jsonData.split("<modelCode>")[m + cc].split("</modelCode>")[0]
                cc = 30
                continue
            else:
                pass
            
    except:
        continue
    else:
        pass

wb.values_append("'P4RAW'!F" + str(int(p4row)+1), {"valueInputOption": "USER_ENTERED"}, {"values": p4table})
wb.values_append("'P1RAW'!F" + str(int(p1row)+1), {"valueInputOption": "USER_ENTERED"}, {"values": p1table})
wb.values_append("'S20RAW'!F" + str(int(s20row)+1), {"valueInputOption": "USER_ENTERED"}, {"values": s20table})
wb.values_append("'S5RAW'!F" + str(int(s5row)+1), {"valueInputOption": "USER_ENTERED"}, {"values": s5table})
wb.values_append("'S10RAW'!F" + str(int(s10row)+1), {"valueInputOption": "USER_ENTERED"}, {"values": s10table})

time.sleep(5)

SPREADSHEET_KEY = '1P5LUFHL6pPQugsjTBBMBWybdLN_36bxkmUvZzYwqMDI' # スプレッドシートのID
gc = gspread.authorize(credentials)
wb = gc.open_by_key(SPREADSHEET_KEY)

worksheet = wb.worksheet("Config")
p4row = worksheet.cell(11,  23).value
p1row = worksheet.cell(11,  24).value
s20row = worksheet.cell(11,  28).value
s5row = worksheet.cell(11,  29).value
s10row = worksheet.cell(11,  30).value

time.sleep(5)

wb.values_append("'P4RAW'!F" + str(int(p4row)+1), {"valueInputOption": "USER_ENTERED"}, {"values": p4table})
wb.values_append("'P1RAW'!F" + str(int(p1row)+1), {"valueInputOption": "USER_ENTERED"}, {"values": p1table})
wb.values_append("'S20RAW'!F" + str(int(s20row)+1), {"valueInputOption": "USER_ENTERED"}, {"values": s20table})
wb.values_append("'S5RAW'!F" + str(int(s5row)+1), {"valueInputOption": "USER_ENTERED"}, {"values": s5table})
wb.values_append("'S10RAW'!F" + str(int(s10row)+1), {"valueInputOption": "USER_ENTERED"}, {"values": s10table})


['08/16', '025665', '1', '39744.0', '7223.0', '28', '7']
['08/16', '025665', '2', '46410.0', '-16249.0', '16', '4']
['08/16', '025665', '3', '41282.0', '-902.0', '25', '6']
['08/16', '025665', '4', '50513.0', '5719.0', '32', '6']
['08/16', '025665', '5', '39744.0', '-29791.0', '4', '2']
['08/16', '025665', '6', '41026.0', '3612.0', '27', '5']
['08/16', '025665', '7', '43333.0', '-18656.0', '17', '8']
['08/16', '025665', '8', '49231.0', '-9328.0', '23', '5']
['08/16', '025665', '9', '40000.0', '-26781.0', '7', '4']
['08/16', '025665', '10', '28974.0', '12038.0', '28', '7']
['08/16', '025665', '11', '43590.0', '-9930.0', '20', '6']
['08/16', '025665', '12', '40513.0', '-28286.0', '6', '2']
['08/16', '025665', '13', '41795.0', '8728.0', '30', '4']
['08/16', '025665', '14', '43077.0', '43033.0', '53', '7']
['08/16', '025665', '15', '41795.0', '27385.0', '42', '5']
['08/16', '025665', '16', '42051.0', '2709.0', '27', '7']
['08/16', '025665', '17', '43333.0', '-12337.0', '20', '9']
['08/16',

['08/16', '025856', '179', '38462.0', '-13842.0', '14', '6']
['08/16', '025856', '180', '52308.0', '4515.0', '35', '9']
['08/16', '025856', '181', '38205.0', '13844.0', '32', '7']
['08/16', '025856', '182', '43590.0', '13242.0', '35', '10']
['08/16', '025856', '183', '32821.0', '25279.0', '36', '2']
['08/16', '025856', '184', '40000.0', '-27082.0', '6', '3']
['08/16', '025856', '185', '34103.0', '-1203.0', '20', '11']
['08/16', '025856', '186', '40513.0', '14445.0', '34', '7']
['08/16', '025856', '187', '35897.0', '-5717.0', '18', '9']
['08/16', '025856', '188', '43846.0', '-15346.0', '16', '6']
['08/16', '025856', '189', '46410.0', '-27082.0', '10', '2']
['08/16', '025856', '190', '38718.0', '34908.0', '46', '11']
['08/16', '025677', '191', '38974.0', '-21064.0', '10', '6']
['08/16', '025677', '192', '35641.0', '-4814.0', '19', '3']
['08/16', '025677', '193', '38205.0', '-20763.0', '8', '6']
['08/16', '025677', '194', '46154.0', '9029.0', '41', '8']
['08/16', '025677', '195', '48205.0

['08/16', '025834', '315', '38974.0', '-14744.0', '10', '5']
['08/16', '025834', '316', '43590.0', '-8124.0', '16', '6']
['08/16', '025834', '317', '58462.0', '3311.0', '30', '9']
['08/16', '025834', '318', '41282.0', '-8726.0', '14', '6']
['08/16', '025842', '319', '38462.0', '-20161.0', '10', '6']
['08/16', '025842', '320', '34103.0', '-2106.0', '19', '5']
['08/16', '025842', '321', '42308.0', '-11434.0', '18', '12']
['08/16', '025885', '322', '27179.0', '9330.0', '99', '6']
['08/16', '025885', '323', '30513.0', '-18055.0', '34', '9']
['08/16', '025839', '324', '21795.0', '-13240.0', '6', '2']
['08/16', '025839', '325', '26923.0', '2408.0', '24', '6']
['08/16', '025839', '326', '31026.0', '6922.0', '25', '3']
['08/16', '025839', '327', '28462.0', '-14444.0', '9', '3']
['08/16', '025839', '328', '19744.0', '-15948.0', '2', '2']
['08/16', '025884', '329', '33590.0', '12640.0', '22', '6']
['08/16', '025884', '330', '36154.0', '-2106.0', '20', '7']
['08/16', '025825', '331', '36410.0', '

['08/16', '025453', '453', '41795.0', '11135.0', '61', '17']
['08/16', '025453', '454', '38974.0', '-1805.0', '44', '16']
['08/16', '025674', '455', '21538.0', '3913.0', '30', '11']
['08/16', '025720', '456', '25385.0', '-3610.0', '34', '9']
['08/16', '025658', '457', '34615.0', '-3610.0', '39', '12']
['08/16', '025747', '458', '17692.0', '22570.0', '47', '6']
['08/16', '025235', '459', '35897.0', '1806.0', '51', '17']
['08/16', '025572', '460', '31282.0', '1806.0', '40', '11']
['08/16', '025744', '461', '26410.0', '8427.0', '45', '7']
['08/16', '025601', '462', '37436.0', '-8124.0', '33', '10']
['08/16', '025735', '463', '26410.0', '-3610.0', '19', '10']
['08/16', '025523', '464', '43846.0', '-1504.0', '49', '20']
['08/16', '025774', '465', '38718.0', '-3911.0', '52', '17']
['08/16', '025790', '466', '25641.0', '-7823.0', '36', '8']
['08/16', '025753', '467', '19231.0', '10533.0', '28', '7']
['08/16', '025785', '468', '20513.0', '5719.0', '27', '8']
['08/16', '025712', '469', '23333.0

['08/16', '120010', '81', '6931', '1852.0', '33', '19', '--']
['08/16', '120010', '81', '6931', '1852.0', '33', '19', '--']
['08/16', '120010', '81', '6931', '1852.0', '33', '19', '--']
['08/16', '120010', '82', '6010', '1296.0', '30', '13', '--']
['08/16', '120010', '82', '6010', '1296.0', '30', '13', '--']
['08/16', '120010', '82', '6010', '1296.0', '30', '13', '--']
['08/16', '120010', '83', '8317', '2315.0', '39', '29', '--']
['08/16', '120010', '83', '8317', '2315.0', '39', '29', '--']
['08/16', '120010', '83', '8317', '2315.0', '39', '29', '--']
['08/16', '120010', '84', '6615', '741.0', '27', '18', '--']
['08/16', '120010', '84', '6615', '741.0', '27', '18', '--']
['08/16', '120010', '84', '6615', '741.0', '27', '18', '--']
['08/16', '120010', '85', '4743', '-602.0', '17', '9', '--']
['08/16', '120010', '85', '4743', '-602.0', '17', '9', '--']
['08/16', '120010', '85', '4743', '-602.0', '17', '9', '--']
['08/16', '120010', '86', '6187', '-1991.0', '17', '11', '--']
['08/16', '12

...

['08/16', '120128', '746', '4261', '10185.0', '0', '1', '27']
['08/16', '120128', '746', '4261', '10185.0', '0', '1', '27']
['08/16', '120128', '746', '4261', '10185.0', '0', '1', '27']
['08/16', '120128', '747', '2866', '-1991.0', '0', '0', '5']
['08/16', '120128', '747', '2866', '-1991.0', '0', '0', '5']
['08/16', '120128', '747', '2866', '-1991.0', '0', '0', '5']
['08/16', '120128', '748', '2458', '-185.0', '0', '1', '8']
['08/16', '120128', '748', '2458', '-185.0', '0', '1', '8']
['08/16', '120128', '748', '2458', '-185.0', '0', '1', '8']
['08/16', '120140', '749', '3569', '2361.0', '0', '2', '112']
['08/16', '120140', '749', '3569', '2361.0', '0', '2', '112']
['08/16', '120140', '749', '3569', '2361.0', '0', '2', '112']
['08/16', '120140', '750', '4152', '4861.0', '0', '4', '125']
['08/16', '120140', '750', '4152', '4861.0', '0', '4', '125']
['08/16', '120140', '750', '4152', '4861.0', '0', '4', '125']
['08/16', '120140', '751', '4141', '3472.0', '0', '2', '57']
['08/16', '120140'

['08/16', '120145', '791', '3775', '-2407.0', '0', '7', '23']
['08/16', '120145', '791', '3775', '-2407.0', '0', '7', '23']
['08/16', '120145', '791', '3775', '-2407.0', '0', '7', '23']
['08/16', '120145', '792', '3089', '-2037.0', '0', '5', '18']
['08/16', '120145', '792', '3089', '-2037.0', '0', '5', '18']
['08/16', '120145', '792', '3089', '-2037.0', '0', '5', '18']
['08/16', '120145', '793', '5684', '-1991.0', '0', '11', '47']
['08/16', '120145', '793', '5684', '-1991.0', '0', '11', '47']
['08/16', '120145', '793', '5684', '-1991.0', '0', '11', '47']
['08/16', '120145', '794', '1331', '-1157.0', '0', '2', '6']
['08/16', '120145', '794', '1331', '-1157.0', '0', '2', '6']
['08/16', '120145', '794', '1331', '-1157.0', '0', '2', '6']
['08/16', '120145', '795', '2874', '-833.0', '0', '3', '20']
['08/16', '120145', '795', '2874', '-833.0', '0', '3', '20']
['08/16', '120145', '795', '2874', '-833.0', '0', '3', '20']
['08/16', '120145', '796', '2197', '-2176.0', '0', '4', '7']
['08/16', '1

['08/16', '120127', '836', '3960', '1481.0', '0', '0', '6']
['08/16', '120127', '836', '3960', '1481.0', '0', '0', '6']
['08/16', '120127', '836', '3960', '1481.0', '0', '0', '6']
['08/16', '120127', '837', '330', '-463.0', '0', '0', '--']
['08/16', '120127', '837', '330', '-463.0', '0', '0', '--']
['08/16', '120127', '837', '330', '-463.0', '0', '0', '--']
['08/16', '120127', '838', '3818', '-1944.0', '0', '0', '6']
['08/16', '120127', '838', '3818', '-1944.0', '0', '0', '6']
['08/16', '120127', '838', '3818', '-1944.0', '0', '0', '6']
['08/16', '120127', '839', '1508', '185.0', '0', '0', '2']
['08/16', '120127', '839', '1508', '185.0', '0', '0', '2']
['08/16', '120127', '839', '1508', '185.0', '0', '0', '2']
['08/16', '120108', '840', '5277', '-1435.0', '0', '0', '28']
['08/16', '120108', '840', '5277', '-1435.0', '0', '0', '28']
['08/16', '120108', '840', '5277', '-1435.0', '0', '0', '28']
['08/16', '120108', '841', '4283', '-1852.0', '0', '0', '20']
['08/16', '120108', '841', '4283

['08/16', '119996', '881', '6263', '4444.0', '36', '28', '--']
['08/16', '119996', '881', '6263', '4444.0', '36', '28', '--']
['08/16', '119996', '881', '6263', '4444.0', '36', '28', '--']
['08/16', '119996', '882', '2297', '-324.0', '8', '5', '--']
['08/16', '119996', '882', '2297', '-324.0', '8', '5', '--']
['08/16', '119996', '882', '2297', '-324.0', '8', '5', '--']
['08/16', '119996', '883', '1943', '-231.0', '7', '3', '--']
['08/16', '119996', '883', '1943', '-231.0', '7', '3', '--']
['08/16', '119996', '883', '1943', '-231.0', '7', '3', '--']
['08/16', '119996', '884', '5560', '-972.0', '17', '14', '--']
['08/16', '119996', '884', '5560', '-972.0', '17', '14', '--']
['08/16', '119996', '884', '5560', '-972.0', '17', '14', '--']
['08/16', '119996', '885', '6012', '972.0', '26', '15', '--']
['08/16', '119996', '885', '6012', '972.0', '26', '15', '--']
['08/16', '119996', '885', '6012', '972.0', '26', '15', '--']
['08/16', '119996', '886', '6719', '3380.0', '39', '17', '--']
['08/16

{'spreadsheetId': '1P5LUFHL6pPQugsjTBBMBWybdLN_36bxkmUvZzYwqMDI',
 'updates': {'spreadsheetId': '1P5LUFHL6pPQugsjTBBMBWybdLN_36bxkmUvZzYwqMDI',
  'updatedRange': 'S10RAW!F7658:M7948',
  'updatedRows': 291,
  'updatedColumns': 8,
  'updatedCells': 2328}}